## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [10]:
!nvidia-smi
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Thu Jul  9 06:40:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    61W / 149W |   7399MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:

def normalize(trainDataArray, testDataArray, mode="MinMax"):
    if mode == "MinMax":
        trainDataArray = (trainDataArray / 255.0)
        testDataArray = (testDataArray / 255.0)
        return (trainDataArray, testDataArray)
    else:
        raise ValueError("Just only one mode: MinMax..." + f"your mode: {mode}")

def toOneHot(trainTargetArray, testTargetArray, target="cifar10"):
    if type(target) == str:
        if target == "cifar10":
            target = 10
    else:
        raise ValueError("somthing was wrong!")
    
    trainTargetArray = keras.utils.to_categorical(trainTargetArray, target)
    testTargetArray = keras.utils.to_categorical(testTargetArray, target)
    return (trainTargetArray, testTargetArray)
    
(x_train, x_test), (y_train, y_test) = keras.datasets.cifar10.load_data()

x_trainN, y_trainN = normalize(x_train, y_train)

x_testOH, y_testOH = toOneHot(x_test, y_test)

In [12]:
def buildModel():
    resnet50 = keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=x_train.shape[1:])
    x = keras.layers.Flatten()(resnet50.output)
    x = keras.layers.Dense(units=256, activation="relu")(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=64, activation="relu")(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=10, activation="softmax")(x)
    model = keras.models.Model(inputs=[resnet50.input], outputs=[x])
    return model

model = buildModel()    
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 18, 18, 64)   0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [15]:
# params
EPOCHS = 100
BATCH_SIZE = 256

model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(x_trainN, x_testOH, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_split=0.25, shuffle=True, callbacks=[keras.callbacks.EarlyStopping(min_delta=1e-4, patience=10, verbose=1)])


Train on 37500 samples, validate on 12500 samples
Epoch 1/100
37500/37500 [==============================] - 77s 2ms/step - loss: 0.6057 - accuracy: 0.8150 - val_loss: 1.2536 - val_accuracy: 0.6813
Epoch 2/100
37500/37500 [==============================] - 62s 2ms/step - loss: 0.6835 - accuracy: 0.8074 - val_loss: 34.6868 - val_accuracy: 0.5666
Epoch 3/100
37500/37500 [==============================] - 61s 2ms/step - loss: 0.5906 - accuracy: 0.8263 - val_loss: 1.4792 - val_accuracy: 0.6506
Epoch 4/100
37500/37500 [==============================] - 62s 2ms/step - loss: 0.4446 - accuracy: 0.8670 - val_loss: 1.0062 - val_accuracy: 0.7347
Epoch 5/100
37500/37500 [==============================] - 62s 2ms/step - loss: 0.3309 - accuracy: 0.9037 - val_loss: 1.1094 - val_accuracy: 0.7310
Epoch 6/100
37500/37500 [==============================] - 62s 2ms/step - loss: 0.3069 - accuracy: 0.9108 - val_loss: 1.0524 - val_accuracy: 0.7505
Epoch 7/100
37500/37500 [==============================] - 61

In [16]:
model.evaluate(y_trainN, y_testOH, verbose=1)

10000/10000 [==============================] - 13s 1ms/step


[1.2540735828399658, 0.7480000257492065]